# PySPTKによる音声の分析再合成 (MLSAフィルタ)

In [11]:
from pysptk.synthesis import MLSADF, Synthesizer
from scipy.io import wavfile
import librosa
import numpy as np
import pysptk
import matplotlib.pyplot as plt
from IPython.display import Audio

In [2]:
# 音声の分析条件
FRAME_LENGTH = 1024 # フレーム長 (point)
HOP_LENGTH = 80 # フレームシフト (point)
MIN_F0 = 60 # 基本周波数の最小値 (Hz)
MAX_F0 = 240 # 基本周波数の最大値 (Hz)
ORDER = 25 # メルケプストラムの分析次数
ALPHA = 0.41 # 周波数ワーピングのパラメタ

In [3]:
IN_WAVE_FILE = "in.wav"       # 入力音声
OUT_WAVE_FILE = "out.wav"     # 分析再合成した音声

In [4]:
# 音声の読み込み
fs, x = wavfile.read(IN_WAVE_FILE)
x = x.astype(np.float64)

In [5]:
# 音声の切り出しと窓掛け
frames = librosa.util.frame(x, frame_length=FRAME_LENGTH,
                            hop_length=HOP_LENGTH).astype(np.float64).T
frames *= pysptk.blackman(FRAME_LENGTH)  # 窓掛け（ブラックマン窓）

In [6]:
# ピッチ抽出
pitch = pysptk.swipe(x, fs=fs, hopsize=HOP_LENGTH,
                     min=MIN_F0, max=MAX_F0, otype="pitch")

In [7]:
# 励振源信号(声帯音源)の生成
source_excitation = pysptk.excite(pitch, HOP_LENGTH)

In [8]:
# メルケプストラム分析（＝スペクトル包絡の抽出）
mc = pysptk.mcep(frames, ORDER, ALPHA)  # メルケプストラム係数の抽出

In [9]:
# メルケプストラム係数からMLSAディジタルフィルタ係数に変換
mlsa_coef = pysptk.mc2b(mc, ALPHA)

In [12]:
# MLSAフィルタの作成
synthesizer = Synthesizer(MLSADF(order=ORDER, alpha=ALPHA), HOP_LENGTH)

In [13]:
# 励振源信号でフィルタを駆動して音声を合成
y = synthesizer.synthesis(source_excitation, mlsa_coef)

In [14]:
# 音声の書き込み
y = y.astype(np.int16)
wavfile.write(OUT_WAVE_FILE, fs, y)

In [ ]:
# オリジナルの音声をプロット
n_samples = len(x)
time = np.arange(n_samples) / fs # 時間軸の設定

# プロット枠を確保 (10がヨコのサイズ、4はタテのサイズ)
plt.figure(figsize=(10, 4))

# x軸のラベル
plt.xlabel("Time (sec)")

# y軸のラベル
plt.ylabel("Amplitude")

# 画像のタイトル
plt.title("Waveform")

# 余白を最小化
plt.tight_layout()

# 波形のプロット
plt.plot(time, x)